# MACS 40200: Problem Set #4

#### By: Jesús Pacheco

In [158]:
%reset
import numpy as np
import scipy.stats as sts
import requests
import scipy.optimize as opt
import numpy.linalg as lin

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


Downloading the data:

In [159]:
#Read the txt file
url_macro = ('https://raw.githubusercontent.com/jesuspachecov/StructEst_W20/' +
       'master/ProblemSets/PS4/data/NewMacroSeries.txt')
response = requests.get(url_macro,allow_redirects=True)

open('MacroSeries.txt', 'wb').write(response.content)
macroseries = np.loadtxt("MacroSeries.txt",
  # dtype={'names': ('ct', 'kt', 'wt', 'rt', 'yt'),
   #       'formats': (np.float, np.float, np.float, np.float)},
   delimiter=',', skiprows=0)
#macroseries[1:10,]

Getting a seeded draw of uniform values that will be used in the simulation:

In [163]:
N = 100
S = 1000
np.random.seed(123)
unif_vals_mat = sts.uniform.rvs(0, 1, size=(N, S))
unif_vals_mat.shape

(100, 1000)

### a) Simulated Method of Moments estimation of macro model

In [164]:
def simulated_data(unif_vals, alpha, mu, rho, sigma, beta=.99):
    '''
    This function simulates the data given the parameters, with the random draw of uniform values. 
    The output are the simulated ct, yt and kt. 
    '''
    sim_err = sts.norm.ppf(unif_vals, loc=0, scale=sigma)
    
    #from eq (5) -> simulate zt
    sim_zt = np.zeros_like(sim_err, dtype=np.float128)
    sim_zt[0,] = mu + sim_err[0,]
    for i in range(1,100):
        sim_zt[i,] = rho * sim_zt[i-1,] + (1-rho)*mu + sim_err[i,]
    
    #from eq (7) -> simulate kt+1
    sim_ktp1 = np.zeros_like(sim_err, dtype=np.float128)
    sim_ktp1[0,] = alpha*beta*(np.exp(sim_zt[0,])) * (np.mean(kt))**alpha
    for i in range(1,100):
        sim_ktp1[i,] = alpha*beta*(np.exp(sim_zt[0,])) * (sim_ktp1[i-1,])**alpha
    sim_kt = np.zeros_like(sim_err)
    sim_kt = np.vstack((np.repeat(np.mean(kt),unif_vals.shape[1]),sim_ktp1[:-1]))
    
    #from eq (3) -> simulate wt
    sim_wt = np.zeros_like(sim_err, dtype=np.float128)
    sim_wt =  (sim_kt**alpha) * (1-alpha) * np.exp(sim_zt)
    
    #from eq (4) -> simulate rt
    sim_rt = alpha * np.exp(sim_zt) * sim_kt**(alpha-1)
    
    #from eq (2) -> simulate ct
    sim_ct = sim_wt + (sim_rt * sim_kt) - sim_ktp1
    sim_ct_1 = np.vstack((np.repeat(np.mean(sim_ct[0]),unif_vals.shape[1]), sim_ct[:-1]))
    
    #from eq (6) -> simulate yt
    sim_yt = np.exp(sim_zt) * sim_kt**alpha

    return sim_ct, sim_ct_1, sim_kt, sim_yt

I tried doing the functions separately but I couldn't get my criterion function to converge, so I'll try with all the functions in the criterion function as: 

In [165]:
def criterion (params, *args):
    alpha, mu, rho, sigma = params
    unif_vals, ct, kt, yt, ct_1, W_I = args
    
    #SIMULATED DATA
    sim_err = sts.norm.ppf(unif_vals, loc=0, scale=sigma)
    #from eq (5) -> simulate zt
    sim_zt = np.zeros_like(sim_err, dtype=np.float128)
    sim_zt[0,] = mu + sim_err[0,]
    for i in range(1,100):
        sim_zt[i,] = rho * sim_zt[i-1,] + (1-rho)*mu + sim_err[i,]
    #from eq (7) -> simulate kt+1
    sim_ktp1 = np.zeros_like(sim_err, dtype=np.float128)
    sim_ktp1[0,] = alpha*beta*(np.exp(sim_zt[0,])) * (np.mean(kt))**alpha
    for i in range(1,100):
        sim_ktp1[i,] = alpha*beta*(np.exp(sim_zt[0,])) * (sim_ktp1[i-1,])**alpha
    sim_kt = np.zeros_like(sim_err)
    sim_kt = np.vstack((np.repeat(np.mean(kt),unif_vals.shape[1]),sim_ktp1[:-1]))
    #from eq (3) -> simulate wt
    sim_wt = np.zeros_like(sim_err, dtype=np.float128)
    sim_wt =  (sim_kt**alpha) * (1-alpha) * np.exp(sim_zt)
    #from eq (4) -> simulate rt
    sim_rt = alpha * np.exp(sim_zt) * sim_kt**(alpha-1)
    #from eq (2) -> simulate ct
    sim_ct = sim_wt + (sim_rt * sim_kt) - sim_ktp1
    sim_ct_1 = np.vstack((np.repeat(np.mean(sim_ct[0]),unif_vals.shape[1]), sim_ct[:-1]))
    #from eq (6) -> simulate yt
    sim_yt = np.exp(sim_zt) * sim_kt**alpha
    
    #DATA MOMENTS
    md1 = ct.mean()
    md2 = kt.mean() 
    md3 = (ct/yt).mean() 
    md4 = yt.var()
    md5 = np.corrcoef(ct, ct_1)[0,1]
    md6 = np.corrcoef(ct, kt)[0,1]
    moms_data = np.array([md1, md2, md3, 
                          md4, md5, md6])

    #SIMULATED DATA MOMENTS
    ms1 = sim_ct.mean(axis=0)
    ms2 = sim_kt.mean(axis=0)
    ms3 = (sim_ct/sim_yt).mean(axis=0)
    ms4 = sim_yt.var(axis=0) 
    ms5 = np.corrcoef(sim_ct, sim_ct_1)[0,1]
    ms6 = np.corrcoef(sim_ct, sim_kt)[0,1]
    moms_model = np.array([np.mean(ms1), np.mean(ms2), np.mean(ms3),
                          np.mean(ms4), np.mean(ms5), np.mean(ms6)])
    
    #ERROR VECTOR
    err =  moms_model - moms_data
    
    #CRITICAL VALUES
    crit_val = err @ W_I  @ err.T
    
    return crit_val

In [166]:
#MINIMIZATION
#Initial guesses and args
params_init= np.array([0.5, 8.92559606, 0.680047, .6]) #From the initial guesses of previous psets

ct = macroseries[:,0]
kt = macroseries[:,1]
yt = macroseries[:,4]
ct_1 = np.append(ct[0], ct[:-1])
beta = .99

W_I = np.eye(6)

smm_args = (unif_vals_mat, ct, kt, yt, ct_1, W_I)

#MINIMIZATION
results_macro = opt.minimize(criterion, params_init, args=(smm_args), method='L-BFGS-B',
                             bounds = ((0.01, 0.9), (None, None), (None, None), (.01, 1.1)))
                             #bounds = ((0.11, 0.99), (5, 14), (-.99, .99), (.01, 1.1)), tol=1e-14)
                             #I played with the boundaries, because with some of them it wasn't converging
alpha_smm, mu_smm, rho_smm, sigma_smm = results_macro.x
params_smm = np.array([alpha_smm, mu_smm, rho_smm, sigma_smm ])
print ("Alpha:", alpha_smm)
print ("Mu:", mu_smm)
print ("Rho:", rho_smm)
print ("Sigma:", sigma_smm)
print ("Minimized criterion function: ", criterion(params_smm, unif_vals_mat, ct, kt, yt, ct_1, W_I))
print ("VCV: ", results_macro.hess_inv.matmat(np.eye(4)))

Alpha: 0.4787835146978975
Mu: 8.413846315160335
Rho: -0.027761949292597458
Sigma: 0.5978985639339959
Minimized criterion function:  59325913966244.597416
VCV:  [[ 0.03598809 -0.46710382 -0.57311065 -0.08615462]
 [-0.46710382  9.01551436 11.35421971 -0.42258974]
 [-0.57311065 11.35421971 17.12377434 -0.62464766]
 [-0.08615462 -0.42258974 -0.62464766  1.01105959]]


### Part b) Two-step weighted matrix

In [168]:
R = 6
S = unif_vals_mat.shape[1]
Err_mat = np.zeros((R, S))

#DATA MOMENTS
md1 = ct.mean()
md2 = kt.mean() 
md3 = (ct/yt).mean() 
md4 = yt.var()
md5 = np.corrcoef(ct, ct_1)[0,1]
md6 = np.corrcoef(ct, kt)[0,1]

#SIMULATED DATA MOMENTS
sim_ct, sim_ct_1, sim_kt, sim_yt = simulated_data(unif_vals_mat, alpha_smm, mu_smm, rho_smm, sigma_smm, beta=.99)
ms1 = sim_ct.mean(axis=0)
ms2 = sim_kt.mean(axis=0)
ms3 = (sim_ct/sim_yt).mean(axis=0)
ms4 = sim_yt.var(axis=0) 
ms5 = np.corrcoef(sim_ct, sim_ct_1)[0,1]
ms6 = np.corrcoef(sim_ct, sim_kt)[0,1]

Err_mat[0, :] = md1 - ms1
Err_mat[1, :] = md2 - ms2
Err_mat[2, :] = md3 - ms3
Err_mat[3, :] = md4 - ms4
Err_mat[4, :] = md5 - ms5
Err_mat[5, :] = md6 - ms6

In [193]:
VCV2 = (1 / macroseries.shape[0]) * (Err_mat @ Err_mat.T)
W_2s = lin.inv(VCV2)
W_2s

array([[ 1.46700265e-13,  1.67973264e-13,  1.31357393e-06,
        -4.10295293e-21, -4.25790671e-07, -7.77531125e-07],
       [ 1.67973264e-13,  2.22395353e-13,  1.77086458e-06,
        -6.86910649e-21, -5.20917058e-07, -9.49202090e-07],
       [ 1.31357393e-06,  1.77086458e-06,  1.66071434e+01,
        -3.18182181e-14, -4.25258420e+00, -7.80618692e+00],
       [-4.10295293e-21, -6.86910649e-21, -3.18182181e-14,
         5.70252992e-28,  1.29251918e-14,  2.28564216e-14],
       [-3.40892416e-07, -4.14330104e-07, -3.34952264e+00,
         1.04873015e-14, -1.39184407e+13,  1.39184180e+13],
       [-8.62388123e-07, -1.05557763e-06, -8.70656932e+00,
         2.52896280e-14,  1.39184180e+13, -1.39183954e+13]])

Now we do the minimization again with this two-step W

In [208]:
smm_args2 = (unif_vals_mat, ct, kt, yt, ct_1, W_2s)

#MINIMIZATION
results_macro2 = opt.minimize(criterion, params_init, args=(smm_args2), method='L-BFGS-B',
                             bounds = ((0.1, 0.9), (None, None), (-.99, None), (.01, 1.1)))
                             #bounds = ((0.11, 0.99), (5, 14), (-.99, .99), (.01, 1.1)), tol=1e-14)
                             #Again, converge only reached with certain boundaries only.
alpha_smm2, mu_smm2, rho_smm2, sigma_smm2 = results_macro2.x
params_smm2 = np.array([alpha_smm2, mu_smm2, rho_smm2, sigma_smm2 ])
print ("Alpha:", alpha_smm2)
print ("Mu:", mu_smm2)
print ("Rho:", rho_smm2)
print ("Sigma:", sigma_smm2)
print ("Minimized criterion function: ", criterion(params_smm2, unif_vals_mat, ct, kt, yt, ct_1, W_2s))

Alpha: 0.5278091969827978
Mu: 7.926920535654525
Rho: 0.6974547786527608
Sigma: 0.43178986032163147
Minimized criterion function:  -18.696468993594862695


The estimation did change quite a bit, specially but rho that is now positive.